In [28]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import Counter

import functools

import math

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

In [36]:
inputFile = "water-treatmennt-original-marked.csv"

dataFrame = pd.read_csv(inputFile, header = 0, sep = ';')
print(dataFrame.shape)
data = dataFrame.values

print('First few data rows:')
print(data[0:10, 0:5])

numSamples = dataFrame.shape[0]
print("Num samples == " + str(numSamples))
numFeatures = dataFrame.shape[1] - 2  #first feature is sampling date and last feature used as class marker
print("Num features == " + str(numFeatures))

(527, 40)
First few data rows:
[['D-1/3/90' 44101.0 1.5 7.8 nan]
 ['D-2/3/90' 39024.0 3.0 7.7 nan]
 ['D-4/3/90' 32229.0 5.0 7.6 nan]
 ['D-5/3/90' 35023.0 3.5 7.9 205.0]
 ['D-6/3/90' 36924.0 1.5 8.0 242.0]
 ['D-7/3/90' 38572.0 3.0 7.8 202.0]
 ['D-8/3/90' 41115.0 6.0 7.8 nan]
 ['D-9/3/90' 36107.0 5.0 7.7 215.0]
 ['D-11/3/90' 29156.0 2.5 7.7 206.0]
 ['D-12/3/90' 39246.0 2.0 7.8 172.0]]
Num samples == 527
Num features == 38


In [48]:
Y = dataFrame.iloc[:, -1]  # Class markers
X = dataFrame.iloc[:, 1:-1]  # Features without date and class markers

print(X.iloc[0:10, 0:2])

# featureMaxValues = [0] * numFeatures
# featureMinValues = [0] * numFeatures

# for featureId in range(numFeatures):
#     featureMaxValues[featureId] = np.amax(X[:, featureId])
#     featureMaxValues[featureId] = np.amin(X[:, featureId])

normalizedX = MinMaxScaler().fit_transform(X)
X = normalizedX

print(X[0:10, 0:2])

   Q-E      (input flow to plant)   ZN-E     (input Zinc to plant)
0                         44101.0                              1.5
1                         39024.0                              3.0
2                         32229.0                              5.0
3                         35023.0                              3.5
4                         36924.0                              1.5
5                         38572.0                              3.0
6                         41115.0                              6.0
7                         36107.0                              5.0
8                         29156.0                              2.5
9                         39246.0                              2.0
[[0.68059803 0.04191617]
 [0.57912095 0.08682635]
 [0.44330515 0.14670659]
 [0.49915053 0.10179641]
 [0.53714697 0.04191617]
 [0.57008655 0.08682635]
 [0.62091503 0.17664671]
 [0.52081709 0.14670659]
 [0.38188323 0.07185629]
 [0.58355819 0.05688623]]


In [ ]:
def addFeatureForBias(data):
    extendedData = np.zeros((data.shape[0], data.shape[1] + 1))
    extendedData[:, 0:-1] = data
    extendedData[:, -1] = int(1)
    
    return extendedData

In [ ]:
def computeSoftMarginCost(W, X, Y):
    # Hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularizationStrength * (np.sum(distances) / N)
    
    # Soft margin cost function
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost